In [19]:
import ast
import os

import pandas as pd

our_dataset_path = '.'

posts_path = os.path.join(our_dataset_path, 'Data\\posts.csv')
fact_checks_path = os.path.join(our_dataset_path, 'Data\\fact_checks.csv')
fact_check_post_mapping_path = os.path.join(our_dataset_path, 'Data\\pairs.csv')

for path in [posts_path, fact_checks_path, fact_check_post_mapping_path]:
    print(f"Checking path: {path}")
    assert os.path.isfile(path), f"File not found: {path}"


Checking path: .\Data\posts.csv
Checking path: .\Data\fact_checks.csv
Checking path: .\Data\pairs.csv


In [20]:
parse_col = lambda s: ast.literal_eval(s.replace('\n', '\\n')) if s else s

df_fact_checks = pd.read_csv(fact_checks_path).fillna('').set_index('fact_check_id')
for col in ['claim', 'instances', 'title']:
    df_fact_checks[col] = df_fact_checks[col].apply(parse_col)


df_posts = pd.read_csv(posts_path).fillna('').set_index('post_id')
for col in ['instances', 'ocr', 'verdicts', 'text']:
    df_posts[col] = df_posts[col].apply(parse_col)


df_fact_check_post_mapping = pd.read_csv(fact_check_post_mapping_path) 